In [1]:
open class Day(private val dayNum: Int) {
    val dataFolder = java.io.File("data21")
    fun readDay(number: Int) = dataFolder.resolve("day$number.txt").readText().lines()
    protected val input by lazy { readDay(dayNum) }
    
    open fun part1(): Any {
        return 0
    }
    
    open fun part2(): Any {
        return 0
    }
    
    fun render(): Any {
        return HTML("""
            <h3 id="day$dayNum">Day $dayNum.</h2>
            <p><b>Part 1 answer:</b> ${part1()}</p>
            <p><b>Part 2 answer:</b> ${part2()}</p>
        """.trimIndent())
    }
}

fun day(number: Int, p1: (List<String>) -> Int, p2: (List<String>) -> Int): Day {
    return object : Day(number) {
        override fun part1(): Int {
            return p1(input)
        }
        override fun part2(): Int {
            return p2(input)
        }
    }
}

USE {
    render<Day> { it.render() }
}

In [2]:
object : Day(0) {
    override fun part1(): Any {
        return 0
    }
    
    override fun part2(): Any {
        return 0
    }
}

Day 0. 
 Part 1 answer: 0 
 Part 2 answer: 0

In [3]:
object : Day(1) {
    val inputList = input.map { it.toInt() }
    fun List<Int>.howMany() = subList(1, size).zip(subList(0, size - 1)).count {(after, before) -> after > before}
    
    override fun part1(): Int {
        return inputList.howMany()
    }
    
    override fun part2(): Int {
        return inputList.windowed(3).map { it.sum() }.howMany()
    }
}

Day 1. 
 Part 1 answer: 1616 
 Part 2 answer: 1645

In [4]:
object : Day(2) {
    override fun part1(): Int {
        var x = 0
        var d = 0

        input.forEach {
            val command = it.substringBefore(' ')
            val value = it.substringAfter(' ').toInt()
            when(command) {
                "up" -> d -= value
                "down" -> d += value
                "forward" -> x += value
            }
        }
        return x * d
    }
    
    override fun part2(): Int {
        var x = 0
        var d = 0
        var aim  = 0
        input.forEach {
            val command = it.substringBefore(' ')
            val value = it.substringAfter(' ').toInt()
            when(command) {
                "up" -> aim -= value
                "down" -> aim += value
                "forward" -> {
                    x += value
                    d += aim * value
                }
            }
        }
        return x * d
    }
}

Day 2. 
 Part 1 answer: 1728414 
 Part 2 answer: 1765720035

In [5]:
object : Day(3) {
    val n = input.first().length
    val m = input.size
    
    fun List<Int>.convert() = fold(0) {acc, b -> acc * 2 + b}
    
    fun List<String>.myFilter(bitI: Int, mostCommon: Boolean): List<String> {
        val n = size
        val r = map { it[bitI] == '1' }
        val onesCount = r.count { it }
        val isOnes = onesCount * 2 >= n
        return filterIndexed { i, _ -> r[i] xor isOnes xor mostCommon  }
    }

    fun List<String>.myFold(mostCommon: Boolean) = foldIndexed(this) { bitI, acc, _ ->
        if (acc.size <= 1) acc else { acc.myFilter(bitI, mostCommon) }
    }.single().map {if(it == '0') 0 else 1}.convert()
    
    override fun part1(): Int {
        val result = input.fold(MutableList(n){0}) { acc, line ->
            line.forEachIndexed { i, c -> if (c == '1') acc[i]++ };
            acc 
        }.map { if (it * 2 >= m) 1 else 0 }
        val iresult = result.map { 1 - it }
        return result.convert() * iresult.convert()
    }
    
    override fun part2(): Int {
        return input.myFold(true) * input.myFold(false)
    }
}

Day 3. 
 Part 1 answer: 2035764 
 Part 2 answer: 2817661

In [6]:
class Board(val data: List<List<Int>>) {
    val mask = MutableList(5) { MutableList(5) { false } }
    
    fun setNum(n: Int) {
        data.forEachIndexed { i, row ->
            row.forEachIndexed { j, num -> 
                if (n == num) mask[i][j] = true
            }
        }
    }
    
    fun check(): Boolean {
        if (mask.any { row -> row.all { it } }) return true
        
        for (i in 0..4) {
            if(mask.all { it[i] }) return true
        }
        
        return false
    }
    
    fun sumUnmarked(): Int {
        return data.mapIndexed { rowI, row -> row.filterIndexed { i, _ -> !mask[rowI][i] } }.flatten().sum()
    }
}

object : Day(4) {
    val requests = input.first().split(",").map {it.toInt()}
    val splitRegex = " +".toRegex()
    val boards = (2 .. (input.size - 2) step 6).map { start ->
        val borderLines = input.subList(start, start + 5)
        // println(borderLines)
        Board(borderLines.map { it.split(splitRegex).filter {!it.isBlank()} .map { it.toInt() } })
    }
    
    override fun part1(): Int {        
        for (r in requests) {
            for (b in boards) {
                b.setNum(r)
                if (b.check()) {
                    return b.sumUnmarked() * r                    
                }
            }
        }
        return 0
    }
    
    override fun part2(): Int {
        val winners = mutableSetOf<Int>()
        var lastScore  = -1
        
        for (r in requests) {
            for ((ib, b) in boards.withIndex()) {
                if (ib in winners) continue
                b.setNum(r)
                if (b.check()) {
                    winners.add(ib)
                    lastScore = b.sumUnmarked() * r                    
                }
            }
        }
        return lastScore
    }
}

Day 4. 
 Part 1 answer: 74320 
 Part 2 answer: 17884

In [7]:
object : Day(5) {
    val n = 1000
    
    inner class Line(
        val x1: Int,
        val y1: Int,
        val x2: Int,
        val y2: Int,
    ) {
        fun points(): List<Pair<Int, Int>> {
            return buildList {
                val mi = (x2 - x1).sign
                val mj = (y2 - y1).sign
                var x = x1
                var y = y1
                val n = kotlin.math.max((x2 - x1).absoluteValue, (y2 - y1).absoluteValue)

                for(i in 0..n) {
                    add(x to y)
                    x += mi
                    y += mj
                }
            }
        }

        val isDiagonal get() = x1 != x2 && y1 != y2
    }
    
    val lines = input.map { line -> 
        val (p1, p2) = line.split(" -> ")
        val (x1, y1) = p1.split(",").map {it.toInt()}
        val (x2, y2) = p2.split(",").map {it.toInt()}
        Line(x1, y1, x2, y2)
    }
    
    fun MutableList<IntArray>.mark(pt: Pair<Int, Int>) {
        val v = this[pt.first][pt.second]
        this[pt.first][pt.second] = if (v == 0) 1 else 2
    }
    
    override fun part1(): Int {
        val a = MutableList(n) { IntArray(n) }
        for(line in lines) {
            if (line.isDiagonal) continue
            line.points().forEach { pt ->
                a.mark(pt)
            }
        }
        return a.sumOf { it.count { it == 2 }}
    }
    
    override fun part2(): Int {
        val a = MutableList(n) { IntArray(n) }
        for(line in lines) {
            line.points().forEach { pt ->
                a.mark(pt)
            }
        }
        return a.sumOf { it.count { it == 2 }}
    }
}

Day 5. 
 Part 1 answer: 4421 
 Part 2 answer: 18674

In [8]:
object : Day(6) {    
    val fishes = input.first().split(",").map { it.toInt() }
    
    override fun part1(): Int {
        var state = fishes
        var born = 0;
        for (i in 1..80) {
            val newFishes = state.map { if (it == 0) 6 else it - 1 }
            state = newFishes + MutableList(born) { 8 }
            born = newFishes.count {it == 0}
        }
        return state.size
    }
    
    override fun part2(): Long {
        var state = MutableList(9) { 0L }
        fishes.forEach { state[it]++ }
        for (i in 1..256) {
            state = MutableList(9) { if (it == 8) state[0] else if(it == 6) state[7] + state[0] else state[it + 1] }
        }
        return state.sum()
    }
}

Day 6. 
 Part 1 answer: 389726 
 Part 2 answer: 1743335992042

In [9]:
object : Day(7) {
    val a = input.first().split(",").map { it.toInt() }
    
    override fun part1(): Any {
        val t = a.sorted()[500]
        println(a.sumOf { kotlin.math.abs(it - t) })
        
        return a.minOf { el -> a.sumOf { kotlin.math.abs(it - el) } }
    }
    
    override fun part2(): Any {
        return a.minOf { el -> a.sumOf { val k = kotlin.math.abs(it - el); k.toLong() * (k + 1)/2 } }
    }
}

355764


Day 7. 
 Part 1 answer: 355764 
 Part 2 answer: 99634572

In [10]:
@file:DependsOn("com.github.shiguruikai:combinatoricskt:1.6.0")

import com.github.shiguruikai.combinatoricskt.*

object : Day(8) {
    val perms = "abcdefg".toCharArray().toList().permutations().map { String(it.toCharArray()) }.toList()
    
    inner class Entry(val signals: List<String>, val displays: List<String>)
    val entries = input.map { 
        val (sig, dis) = it.split("|")
        Entry(sig.trim().split(" "), dis.trim().split(" "))
    }
    
    fun String.decode(arg: String): Int? {
        val res = CharArray(arg.length)
        for (i in 0 until arg.length) {
            res[i] = this[arg[i].code - 'a'.code]
        }
        res.sort()
        
        return when(String(res)) {
            "abcefg" -> 0
            "cf" -> 1
            "acdeg" -> 2
            "acdfg" -> 3
            "bcdf" -> 4
            "abdfg" -> 5
            "abdefg" -> 6
            "acf" -> 7
            "abcdefg" -> 8
            "abcdfg" -> 9
            else -> null
        }
    }
    
    override fun part1(): Int {
        return entries.sumOf { it.displays.count { when(it.length) { 2, 3, 4, 7 -> true; else -> false } } }
    }
    
    override fun part2(): Long {
        return entries.sumOf { entry ->
            val p = perms.first { perm -> entry.signals.all { signal -> perm.decode(signal) != null } }
            entry.displays.map { p.decode(it)!!.toString() }.joinToString("").toLong()
        }
    }
}

Day 8. 
 Part 1 answer: 532 
 Part 2 answer: 1011284

In [11]:
object : Day(9) {
    val mat = input.map { line -> line.toCharArray().map { it.code - '0'.code } }
    val n = mat.size
    val m = mat.first().size
    
    override fun part1(): Any {
        var risk = 0
        for (i in 0 until n) {
            for (j in 0 until m) {
                val el = mat[i][j]
                var cond = !(
                    i > 0 && mat[i - 1][j] <= el ||
                    j > 0 && mat[i][j - 1] <= el ||
                    i < n - 1 && mat[i + 1][j] <= el ||
                    j < m - 1 && mat[i][j + 1] <= el
                )
                if (cond) risk += el + 1
            }
        }
        
        return risk
    }
    
    override fun part2(): Any {
        val basins = mutableListOf<Long>()
        val mask = MutableList(n) { i -> MutableList(m) { j -> mat[i][j] == 9 } }
        
        fun dfs(i: Int, j: Int): Int {
            if (i < 0 || j < 0 || i >= n || j >= m || mask[i][j]) return 0
            
            mask[i][j] = true;
            return 1 + dfs(i - 1, j) + dfs(i, j - 1) + dfs(i + 1, j) + dfs(i, j + 1)
        }
        
        for (i in 0 until n) {
            for (j in 0 until m) {
                basins.add(dfs(i, j).toLong())
            }
        }
        
        basins.sortDescending()
        return basins[0] * basins[1] * basins[2]
    }
}

Day 9. 
 Part 1 answer: 560 
 Part 2 answer: 959136

In [12]:
object : Day(10) {
    val Char.isOpen get() = when(this) { '(', '{', '[', '<' -> true; else -> false }
    val Char.openPair get() = when(this) {
        ')' -> '('
        '}' -> '{'
        ']' -> '['
        '>' -> '<'
        else -> error("Not a closing bracket: $this")
    }
    
    fun count1(line: String): Long {
        val st = MutableList<Char>(0) { 'a' }
        for (c in line) {
            when {
                c.isOpen -> st.add(c)
                else -> {
                    if (st.lastOrNull() == c.openPair) st.removeLast()
                    else return when(c) {
                        ')' -> 3
                        ']' -> 57
                        '}' -> 1197
                        '>' -> 25137
                        else -> error("Not a closing bracket: $c")
                    }
                }
            }
        }
        return 0
    }
    
    fun count2(line: String): Long {
        val st = MutableList<Char>(0) { 'a' }
        for (c in line) {
            when {
                c.isOpen -> st.add(c)
                else -> {
                    if (st.lastOrNull() == c.openPair) st.removeLast()
                    else return 0
                }
            }
        }
        
        val comString = st.reversed()
        
        return comString.fold(0) { score, el -> 
            score * 5 + when(el) {
                '(' -> 1
                '[' -> 2
                '{' -> 3
                '<' -> 4
                else -> error("Impossible state")
            }
        }
    }
    
    override fun part1(): Any {
        var score = 0L
        for (line in input) {
            score += count1(line)
        }
        return score
    }
    
    override fun part2(): Long {
        val ls = buildList {
            for (line in input) {
                val s = count2(line)
                if(s != 0L) add(s)
            }
        }        
        return ls.sorted()[ls.size/2]
    }
}

Day 10. 
 Part 1 answer: 469755 
 Part 2 answer: 2762335572

In [13]:
object : Day(11) {
    val pairs = listOf(0 to 1, 0 to -1, 1 to 0, -1 to 0, 1 to 1, -1 to 1, 1 to -1, -1 to -1)
    val n = 10
    fun getMap() = input.map { it.toCharArray().map { it.code - '0'.code }.toMutableList() }.toMutableList()
    
    fun MutableList<MutableList<Int>>.inc(i: Int, j: Int) {
        for (p in pairs) {
            val s = i + p.first
            val t = j + p.second
            if (s >= 0 && s < n && t >= 0 && t < n) {
                this[s][t]++;
            }
        }
    }
    
    override fun part1(): Any {
        val a = getMap()
        val steps = 100
        var totalFlashes = 0
        val flashMap = MutableList(n) {MutableList(n) {false}}
        
        for (step in 1..steps) {
            for (i in 0 until n) {
                for (j in 0 until n) {
                    a[i][j]++;
                }
            }
            
            var thisFlashes: Int
            do {
                thisFlashes = 0
                for (i in 0 until n) {
                    for (j in 0 until n) {
                        if(a[i][j] > 9 && !flashMap[i][j]) {
                            flashMap[i][j] = true
                            ++totalFlashes
                            ++thisFlashes
                            a.inc(i, j)
                        }
                    }
                }                
            } while (thisFlashes > 0)
            
            for (i in 0 until n) {
                for (j in 0 until n) {
                    if(flashMap[i][j]){
                        flashMap[i][j] = false
                        a[i][j] = 0
                    }
                }
            }
        }
        
        return totalFlashes
    }
    
    override fun part2(): Any {
        val a = getMap()
        var totalFlashes = 0
        val flashMap = MutableList(n) {MutableList(n) {false}}
        
        var step = 0
        while (true) {
            ++step
            
            for (i in 0 until n) {
                for (j in 0 until n) {
                    a[i][j]++;
                }
            }
            
            var thisFlashes: Int
            do {
                thisFlashes = 0
                for (i in 0 until n) {
                    for (j in 0 until n) {
                        if(a[i][j] > 9 && !flashMap[i][j]) {
                            flashMap[i][j] = true
                            ++totalFlashes
                            ++thisFlashes
                            a.inc(i, j)
                        }
                    }
                }                
            } while (thisFlashes > 0)
            
            var d = 0
            for (i in 0 until n) {
                for (j in 0 until n) {
                    if(flashMap[i][j]){
                        ++d
                        flashMap[i][j] = false
                        a[i][j] = 0
                    }
                }
            }
            
            if (d == n * n) {
                return step
            }
        }
    }
}

Day 11. 
 Part 1 answer: 1652 
 Part 2 answer: 220

In [14]:
object : Day(12) {
    inner class Vertex(
        val isSmall: Boolean,
        val isStart: Boolean,
        val isEnd: Boolean,
        val index: Int,
        val name: String,
        val adj: MutableList<Vertex> = mutableListOf()
    )

    inner class Path(
        val last: Vertex,
        val parent: Path? = null,
        val isSpoiled: Boolean = false,
    ) {
        val hc: Int

        init {
            var result = last.index
            result = 31 * result + (parent?.hc ?: 0)
            result = 31 * result + isSpoiled.hashCode()
            hc = result
        }

        override fun hashCode(): Int {
            return hc
        }


        operator fun contains(v: Vertex): Boolean {
            return v === last || parent?.contains(v) == true
        }

        override fun equals(other: Any?): Boolean {
            if (other !is Path) return false
            return last === other.last && parent === other.parent
        }    
    }
    
    
    val nameToVertex = mutableMapOf<String, Vertex>()
    val vertices = mutableListOf<Vertex>()
    val endIndex: Int
    
    init {
        var vertIndex = 0
        
        fun addOrCreate(name: String): Vertex {
            return nameToVertex.getOrPut(name) {
                val v = Vertex(name.first().isLowerCase(), name  == "start", name == "end", vertIndex, name)
                vertIndex++
                vertices.add(v)
                v
            }
        }
        
        for (line in input) {
            val (n1, n2) = line.split("-")
            val v1 = addOrCreate(n1)
            val v2 = addOrCreate(n2)
            v1.adj.add(v2)
            v2.adj.add(v1)
        }
        
        endIndex = vertices.first { it.isEnd }.index
    }
    
    override fun part1(): Any {
        val paths = MutableList(vertices.size) {
            buildList { 
                if(vertices[it].isStart) add(Path(vertices[it]))
            }.toMutableSet()
        }
        
        while (true) {
            var added = false
            for (v in vertices) {
                for (u in v.adj) {
                    if (u.isEnd) continue
                    val pt = paths[u.index]
                    for (p in pt) {
                        if (v.isSmall && v in p) continue                        
                        val newP = Path(v, p, false)
                        if (newP in paths[v.index]) continue
                        paths[v.index].add(newP)
                        added = true
                    }
                }
            }
            if (!added) {
                return paths[endIndex].size
            }
        }
    }
    
    override fun part2(): Any {
        val paths = MutableList(vertices.size) {
            buildList { 
                if(vertices[it].isStart) add(Path(vertices[it]))
            }.toMutableSet()
        }
        
        while (true) {
            var added = false
            for (v in vertices) {
                for (u in v.adj) {
                    if (u.isEnd) continue
                    val pt = paths[u.index]
                    for (p in pt) {
                        if ((v.isStart || v.isEnd) && v in p) continue
                        
                        var newSpoiled = p.isSpoiled
                        if (v.isSmall && v in p) {
                            if (p.isSpoiled) {
                                continue
                            } else {
                                newSpoiled = true
                            }
                        }
                        
                        val newP = Path(v, p, newSpoiled)
                        if (newP in paths[v.index]) continue
                        paths[v.index].add(newP)
                        added = true
                    }
                }
            }
            if (!added) {
                return paths[endIndex].size
            }
        }
    }
}

Day 12. 
 Part 1 answer: 5104 
 Part 2 answer: 149220

In [15]:
data class Pt(val x: Int, val y: Int)

object : Day(13) {
    inner class Fold(val along: Char, val num: Int)
    
    val points: List<Pt>
    val folds: List<Fold>
    
    fun makeFold(pts: Collection<Pt>, fold: Fold): Collection<Pt> {
        return HashSet<Pt>().apply { 
            if (fold.along == 'x') {
                for (p in pts) {
                    if (p.x > fold.num) {
                        add(Pt(2 * fold.num - p.x, p.y))
                    } else {
                        add(p)
                    }
                }
            } else {
                for (p in pts) {
                    if (p.y > fold.num) {
                        add(Pt(p.x, 2 * fold.num - p.y))
                    } else {
                        add(p)
                    }
                }
            }
        }
    }
    
    init {
        var i = 0
        points = buildList {
            for (line in input) {
                ++i
                if (line.isBlank()) break

                val (x, y) = line.split(",")
                add(Pt(x.toInt(), y.toInt()))
            }
        }
        
        folds = buildList {
            while(i < input.size) {
                val line = input[i]
                val suff = line.substringAfter("fold along ")
                val (c, num) = suff.split("=")
                add(Fold(c[0], num.toInt()))
                ++i
            }
        }
    }
    
    
    override fun part1(): Any {
        return makeFold(points, folds[0]).size
    }
    
    override fun part2(): Any {
        val newPoints = folds.fold<Fold, Collection<Pt>>(points) { acc, f -> makeFold(acc, f) }
        
        val minX = newPoints.minOf { it.x }
        val maxX = newPoints.maxOf { it.x }
        val minY = newPoints.minOf { it.y }
        val maxY = newPoints.maxOf { it.y }
        
        for (y in minY..maxY) {
            for (x in minX..maxX) {
                if (Pt(x, y) in newPoints) {
                    print('#')
                } else {
                    print('.')
                }
            }
            print("\n")
        }
        
        return "PERCGJPB"
    }
}

###..####.###...##...##....##.###..###.
#..#.#....#..#.#..#.#..#....#.#..#.#..#
#..#.###..#..#.#....#.......#.#..#.###.
###..#....###..#....#.##....#.###..#..#
#....#....#.#..#..#.#..#.#..#.#....#..#
#....####.#..#..##...###..##..#....###.


Day 13. 
 Part 1 answer: 781 
 Part 2 answer: PERCGJPB

In [16]:
object : Day(14) {
    val initialString = input[0]
    val mapping = input.subList(2, input.size).map { 
        val(s, t) = it.split(" -> ")
        s to t[0]
    }.toMap()
    
    fun <K> MutableMap<K, Long>.inc(key: K, value: Long = 1) {
        this[key] = value + (get(key) ?: 0)
    }
    
    fun String.toPairsMap(): Map<String, Long> {
        val res = mutableMapOf<String, Long>()
        for (i in 0 until (length - 1)) {
            res.inc(substring(i, i + 2))
        }
        return res
    }
    
    fun performInsertion(a: Map<String, Long>): Map<String, Long> {
        val res = mutableMapOf<String, Long>()
        
        for ((key, num) in a) {
            val c = mapping[key]
            if (c != null) {
                res.inc("${key[0]}$c", num)
                res.inc("$c${key[1]}", num)
            } else {
                res.inc(key, num)
            }
        }
        
        return res
    }
    
    fun charCounts(a: Map<String, Long>, beg: Char, end: Char): Map<Char, Long> {
        val res = mutableMapOf<Char, Long>()
        
        res.inc(beg)
        res.inc(end)
        
        for ((key, num) in a) {
            for (c in key) {
                res.inc(c, num)
            }
        }
        
        for ((key, num) in res) {
            res[key] = num / 2
        }
        
        return res
    }
    
    fun Map<Char, Long>.minMaxDiff(): Long {
        val minC = values.minOrNull()!!
        val maxC = values.maxOrNull()!!
        return maxC - minC
    }
    
    override fun part1(): Any {
        var pairs = initialString.toPairsMap()
        for (i in 1..10) {
            pairs = performInsertion(pairs)
        }
        return charCounts(pairs, initialString.first(), initialString.last()).minMaxDiff()
    }
    
    override fun part2(): Any {
        var pairs = initialString.toPairsMap()
        for (i in 1..40) {
            pairs = performInsertion(pairs)
        }
        return charCounts(pairs, initialString.first(), initialString.last()).minMaxDiff()
    }
}

Day 14. 
 Part 1 answer: 2549 
 Part 2 answer: 2516901104210

In [17]:
import java.util.PriorityQueue

object : Day(15) {
    val mtx = input.map { it.map { c -> c.toString().toInt() } }
    val shifts = listOf(-1 to 0, 1 to 0, 0 to 1, 0 to -1)
    
    fun dijkstra(a: List<List<Int>>): Int {
        val n = a.size
        val m = a.first().size
        val res = MutableList(n) { MutableList(m) { Int.MAX_VALUE / 2 } }
        val used = MutableList(n) { MutableList(m) { false } }
        val q = PriorityQueue<Pair<Int, Pair<Int, Int>>> { x, y ->
            x.first - y.first
        }
        
        res[0][0] = 0
        q.add(0 to (0 to 0))
        while(q.isNotEmpty()) {
            val (_, pt) = q.poll()!!
            val (i, j) = pt
            if (used[i][j]) continue

            used[i][j] = true

            for (shift in shifts) {
                val ni = i + shift.first
                val nj = j + shift.second
                if(ni >= 0 && nj >= 0 && ni < n && nj < m) {
                    res[ni][nj] = minOf(res[ni][nj], res[i][j] + a[ni][nj])
                    if (!used[ni][nj]) {
                        q.add(res[ni][nj] to (ni to nj))
                    }
                }
            }
        }

        return res[n-1][m-1]
    }
    
    override fun part1(): Any {
        return dijkstra(mtx)
    }
    
    override fun part2(): Any {
        val n = mtx.size
        val m = mtx.first().size
        val N = n * 5
        val M = m * 5
        val mtx2 = MutableList(N) {MutableList(M) { 0 }}
        
        for (i in 0 until N) {
            for (j in 0 until M) {
                val tileI = i / n
                val tileJ = j / m
                val origI = i % n
                val origJ = j % m
                mtx2[i][j] = (mtx[origI][origJ] - 1 + tileI + tileJ + 9) % 9 + 1
            }
        }
        
        return dijkstra(mtx2)
    }
}

Day 15. 
 Part 1 answer: 373 
 Part 2 answer: 2868

In [18]:
object : Day(16) {
    fun binList(s: String): List<Int> {
        var num = s.toInt(radix = 16)
        val res = MutableList(4) { 0 }
        for (i in 3 downTo 0) {
            res[i] = num % 2
            num /= 2
        }
        return res
    }
    val rawMessage = input.first().flatMap { binList(it.toString()) }
    
    
    fun parseNum(a: List<Int>, start: Int, end: Int): Long {
        var r = 0L
        for (i in start until end) {
            r = r * 2 + a[i]
        }
        return r
    }
    
    inner open class Packet(
        val version: Int
    )
    
    inner class Literal(
        version: Int,
        val num: Long,
    ): Packet(version)
    
    inner class Operator(
        version: Int,
        val children: List<Packet>,
        val type: Int,
    ): Packet(version)
    
    fun parse(a: List<Int>, start: Int): Pair<Int, Packet> {
        val version = parseNum(a, start, start + 3).toInt()
        val type = parseNum(a, start + 3, start + 6).toInt()
        
        var contentStart = start + 6
        var result = if (type == 4) {
            var endBit: Long
            var r = 0L
            
            do {
                endBit = parseNum(a, contentStart, contentStart + 1)
                val v = parseNum(a, contentStart + 1, contentStart + 5)
                r = r * 16 + v
                contentStart += 5
            } while (endBit == 1L)
            
            Literal(version, r)
        } else {
            val iType = parseNum(a, contentStart, contentStart + 1)
            val children = mutableListOf<Packet>()
            ++contentStart
            
            if (iType == 0L) {
                val len = parseNum(a, contentStart, contentStart + 15).toInt()
                contentStart += 15
                var curLen = 0
                while(len > curLen) {
                    val (i, p) = parse(a, contentStart + curLen)
                    curLen = i - contentStart
                    children.add(p)
                }
                contentStart += len
            } else {
                val cnt = parseNum(a, contentStart, contentStart + 11).toInt()
                contentStart += 11
                for (j in 0 until cnt) {
                    val (i, p) = parse(a, contentStart)
                    contentStart = i
                    children.add(p)
                }
            }
            
            Operator(version, children, type)
        }
        
        return contentStart to result
    }
    
    fun sumVersions(p : Packet): Int {
        var r = p.version
        if (p is Operator) {
            for (c in p.children) {
                r += sumVersions(c)
            }
        }
        return r
    }
    
    fun calc(p: Packet): Long {
        if (p is Literal) return p.num
        else if (p is Operator) {
            return when(p.type) {
                0 -> p.children.fold(0L) {acc, el -> acc + calc(el)}
                1 -> p.children.fold(1L) {acc, el -> acc * calc(el)}
                2 -> p.children.fold(Long.MAX_VALUE) {acc, el -> minOf(acc, calc(el))}
                3 -> p.children.fold(Long.MIN_VALUE) {acc, el -> maxOf(acc, calc(el))}
                5 -> if (calc(p.children[0]) > calc(p.children[1])) 1 else 0
                6 -> if (calc(p.children[0]) < calc(p.children[1])) 1 else 0
                7 -> if (calc(p.children[0]) == calc(p.children[1])) 1 else 0
                else -> error("Impossible type: ${p.type}")
            }
        }
        error("Impossible situation")
    }
    
    override fun part1(): Any {
        val (_, p) = parse(rawMessage, 0)
        return sumVersions(p)
    }
    
    override fun part2(): Any {
        val (_, p) = parse(rawMessage, 0)
        return calc(p)
    }
}

Day 16. 
 Part 1 answer: 979 
 Part 2 answer: 277110354175

In [19]:
import kotlin.math.sqrt

object : Day(17) {
    val x1: Int
    val x2: Int
    val y1: Int
    val y2: Int
    
    init {
        val (xs, ys) = input.first().substringAfter("target area: ").split(", ")
        val (x1s, x2s) = xs.substringAfter("x=").split("..")
        val (y1s, y2s) = ys.substringAfter("y=").split("..")
        x1 = x1s.toInt()
        x2 = x2s.toInt()
        y1 = y1s.toInt()
        y2 = y2s.toInt()
    }
    
    fun vxRange() = (sqrt(2 * x1.toDouble()).toInt() - 1) .. x2
    fun vyRange() = -200 .. 500
    
    fun maxHeight(vx0: Int, vy0: Int): Int? {
        var vx = vx0
        var vy = vy0
        var x = 0
        var y = 0
        var maxH = y
        
        while (vy >= 0 || y >= y1) {
            if (x in x1 ..x2 && y in y1..y2) return maxH
            x += vx
            y += vy
            vx = if (vx > 0) vx - 1 else if (vx < 0) vx + 1 else 0
            --vy
            if (y > maxH) maxH = y
        }
        
        return null
    }
    
    override fun part1(): Any {
        var m = 0
        for (vx in vxRange()) {
            for (vy in vyRange()) {
                val h = maxHeight(vx, vy)
                if (h != null && h > m) {
                    m = h
                }
            }
        }
        return m
    }
    
    override fun part2(): Any {
        var c = 0
        for (vx in vxRange()) {
            for (vy in vyRange()) {
                if (maxHeight(vx, vy) != null) ++c
            }
        }
        return c
    }
}

Day 17. 
 Part 1 answer: 7503 
 Part 2 answer: 3229

In [20]:
abstract class FishNumber(var p: FishPair?) {
    val parent: FishPair? get() = p
    
    abstract val magnitude: Long
    
    abstract fun explodable(depth: Int): FishNumber?
    abstract fun explode(node: FishNumber): FishNumber
    abstract fun splittable(): FishNumber?
    abstract fun split(): FishNumber
    abstract fun copy(newP: FishPair?): FishNumber
}

class FishLiteral(var n: Int, parent: FishPair?): FishNumber(parent) {
    override val magnitude get() = n.toLong()
    override fun toString() = n.toString()
    override fun explodable(depth: Int) = null
    override fun splittable() = if (n > 9) this else null
    override fun explode(node: FishNumber) = parent?.explode(node) ?: this
    override fun split(): FishNumber {
        if (n <= 9) return parent?.split() ?: this
        
        val np = FishPair(null, null, parent)
        val a = FishLiteral(n / 2, np)
        val b = FishLiteral(n / 2 + (n % 2), np)
        np.a = a
        np.b = b
        
        if (parent != null) {
            if (this === parent!!.left) parent!!.a = np
            else parent!!.b = np
        }
        
        return np.parent?.split() ?: np
    }
    override fun copy(newP: FishPair?): FishLiteral = FishLiteral(n, newP)
}

class FishPair(var a: FishNumber?, var b: FishNumber?, parent: FishPair?): FishNumber(parent) {
    
    val left: FishNumber get() = a!!
    val right: FishNumber get() = b!!
    
    override val magnitude get() = 3 * left.magnitude + 2 * right.magnitude
    override fun toString() = "[$a,$b]"
    override fun explodable(depth: Int): FishNumber? {
        if (depth >= 4) return this
        val leftR = left.explodable(depth + 1)
        if (leftR != null) return leftR
        return right.explodable(depth + 1)
    }
    override fun splittable(): FishNumber? {
        val leftR = left.splittable()
        if (leftR != null) return leftR
        return right.splittable()
    }
    
    fun leftNeighbour(): FishLiteral? {
        var p = parent
        var c: FishNumber = this
        while(p != null && p.right !== c) {
            c = p
            p = c.parent
        }
        if (p == null) return null
        
        c = p.left
        while (c is FishPair) c = c.right
        return c as FishLiteral
    }
    
    fun rightNeighbour(): FishLiteral? {
        var p = parent
        var c: FishNumber = this
        while(p != null && p!!.left !== c) {
            c = p!!
            p = c.parent
        }
        if (p == null) return null
        
        c = p!!.right
        while (c is FishPair) c = (c as FishPair).left
        return c as FishLiteral
    }
    
    override fun explode(node: FishNumber): FishNumber {
        if (node !== this) return parent?.explode(node) ?: this
        leftNeighbour()?.let { it.n += (left as FishLiteral).n }
        rightNeighbour()?.let { it.n += (right as FishLiteral).n }
        val np = FishLiteral(0, parent)
        if (parent != null) {
            if (this === parent!!.left) parent!!.a = np
            else parent!!.b = np
        }
        return np.explode(node)
    }
    override fun split(): FishNumber = parent?.split() ?: this
    
    override fun copy(newP: FishPair?): FishPair {
        val node = FishPair(null, null, newP)
        node.a = left.copy(node)
        node.b = right.copy(node)
        return node
    }
}

object : Day(18) {
    val inputNumbers = input.map { parseNumber(it, 0, null).first }
    
    fun parseNumber(s: String, start: Int, parent: FishPair?): Pair<FishNumber, Int> {
        if (s[start] == '[') {
            val res = FishPair(null, null, parent)
            val (n1, end1) = parseNumber(s, start + 1, res)
            val (n2, end2) = parseNumber(s, end1 + 1, res)
            res.a = n1
            res.b = n2
            return res to (end2 + 1)
        } else {
            return FishLiteral(s[start].code - '0'.code, parent) to (start + 1)
        }
    }
    
    fun reduceNumber(num: FishNumber): FishNumber {
        var res = num
        while(true) {
            var changed = false
            
            res.explodable(0)?.let {
                changed = true
                res = it.explode(it)
            }
            if (changed) continue
            
            res.splittable()?.let { 
                changed = true
                res = it.split()
            }
            if (changed) continue
            
            break
        }
        return res
    }
    
    fun addNumbers(a: FishNumber, b: FishNumber): FishNumber {
        val n1 = a.copy(null)
        val n2 = b.copy(null)
        val raw = FishPair(n1, n2, null)
        n1.p = raw
        n2.p = raw
        val res = reduceNumber(raw)        
        return res
    }
    
    override fun part1(): Any {
        val res = inputNumbers.reduce { acc, n -> addNumbers(acc, n) }
        return res.magnitude
    }
    
    override fun part2(): Any {
        var mag = 0L
        for (i in 0 until inputNumbers.size) {
            for (j in 0 until inputNumbers.size) {
                if (i == j) continue
                
                mag = maxOf(mag, addNumbers(inputNumbers[i], inputNumbers[j]).magnitude)
            }
        }
        
        return mag
    }
}

Day 18. 
 Part 1 answer: 3699 
 Part 2 answer: 4735